In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import os
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import homogeneity_score
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

In [2]:
df_metrics = pd.DataFrame(columns=['ARI_Louvain','ARI_kmeans','ARI_HC',
                                   'AMI_Louvain','AMI_kmeans','AMI_HC',
                                   'Homogeneity_Louvain','Homogeneity_kmeans','Homogeneity_HC'])

In [3]:
workdir = './output/'
path_fm = os.path.join(workdir,'feature_matrices/')
path_clusters = os.path.join(workdir,'clusters/')
path_metrics = os.path.join(workdir,'metrics/')
os.system('mkdir -p '+path_clusters)
os.system('mkdir -p '+path_metrics)

0

In [4]:
metadata = pd.read_csv('./input/metadata.tsv',sep='\t',index_col=0)
num_clusters = len(np.unique(metadata['label']))
print(num_clusters)

13


In [5]:
files = [x for x in os.listdir(path_fm) if x.startswith('FM')]
len(files)

17

In [6]:
files

['FM_ChromVAR_cusanovich2018subset_kmers.rds',
 'FM_ChromVAR_cusanovich2018subset_motifs.rds',
 'FM_cisTopic_cusanovich2018subset.rds',
 'FM_SnapATAC_cusanovich2018subset.rds',
 'FM_SCRAT_cusanovich2018subset_motifs.rds',
 'FM_BROCKMAN_cusanovich2018subset.rds',
 'FM_Cusanovich2018_cusanovich2018subset.rds',
 'FM_Control_cusanovich2018subset.rds',
 'FM_GeneScoring_cusanovich2018subset.rds',
 'FM_Scasat_cusanovich2018subset.rds',
 'FM_scABC_cusanovich2018subset.rds',
 'FM_Cicero_cusanovich2018subset.rds',
 'FM_ChromVAR_cusanovich2018subset_kmers_pca.rds',
 'FM_ChromVAR_cusanovich2018subset_motifs_pca.rds',
 'FM_GeneScoring_cusanovich2018subset_pca.rds',
 'FM_Cicero_cusanovich2018subset_pca.rds',
 'FM_SCRAT_cusanovich2018subset_pca.rds']

In [7]:
def getNClusters(adata,n_cluster,range_min=0,range_max=3,max_steps=20):
    this_step = 0
    this_min = float(range_min)
    this_max = float(range_max)
    while this_step < max_steps:
        print('step ' + str(this_step))
        this_resolution = this_min + ((this_max-this_min)/2)
        sc.tl.louvain(adata,resolution=this_resolution)
        this_clusters = adata.obs['louvain'].nunique()
        
        print('got ' + str(this_clusters) + ' at resolution ' + str(this_resolution))
        
        if this_clusters > n_cluster:
            this_max = this_resolution
        elif this_clusters < n_cluster:
            this_min = this_resolution
        else:
            return(this_resolution, adata)
        this_step += 1
    
    print('Cannot find the number of clusters')
    print('Clustering solution from last iteration is used:' + str(this_clusters) + ' at resolution ' + str(this_resolution))

In [8]:
for file in files:
    file_split = file.split('_')
    method = file_split[1]
    dataset = file_split[2].split('.')[0]
    if(len(file_split)>3):
        method = method + '_' + '_'.join(file_split[3:]).split('.')[0]
    print(method)

    pandas2ri.activate()
    readRDS = robjects.r['readRDS']
    df_rds = readRDS(os.path.join(path_fm,file))
    fm_mat = pandas2ri.ri2py(robjects.r['data.frame'](robjects.r['as.matrix'](df_rds)))
    fm_mat.fillna(0,inplace=True)
    fm_mat.columns = metadata.index
    
    adata = sc.AnnData(fm_mat.T)
    adata.var_names_make_unique()
    adata.obs = metadata.loc[adata.obs.index,]
    df_metrics.loc[method,] = ""
    #Louvain
    sc.pp.neighbors(adata, n_neighbors=15,use_rep='X')
#     sc.tl.louvain(adata)
    getNClusters(adata,n_cluster=num_clusters)
    #kmeans
    kmeans = KMeans(n_clusters=num_clusters, random_state=2019).fit(adata.X)
    adata.obs['kmeans'] = pd.Series(kmeans.labels_,index=adata.obs.index).astype('category')
    #hierachical clustering
    hc = AgglomerativeClustering(n_clusters=num_clusters).fit(adata.X)
    adata.obs['hc'] = pd.Series(hc.labels_,index=adata.obs.index).astype('category')
    #clustering metrics
    
    #adjusted rank index
    ari_louvain = adjusted_rand_score(adata.obs['label'], adata.obs['louvain'])
    ari_kmeans = adjusted_rand_score(adata.obs['label'], adata.obs['kmeans'])
    ari_hc = adjusted_rand_score(adata.obs['label'], adata.obs['hc'])
    #adjusted mutual information
    ami_louvain = adjusted_mutual_info_score(adata.obs['label'], adata.obs['louvain'],average_method='arithmetic')
    ami_kmeans = adjusted_mutual_info_score(adata.obs['label'], adata.obs['kmeans'],average_method='arithmetic')   
    ami_hc = adjusted_mutual_info_score(adata.obs['label'], adata.obs['hc'],average_method='arithmetic')
    #homogeneity
    homo_louvain = homogeneity_score(adata.obs['label'], adata.obs['louvain'])
    homo_kmeans = homogeneity_score(adata.obs['label'], adata.obs['kmeans'])
    homo_hc = homogeneity_score(adata.obs['label'], adata.obs['hc'])

    df_metrics.loc[method,['ARI_Louvain','ARI_kmeans','ARI_HC']] = [ari_louvain,ari_kmeans,ari_hc]
    df_metrics.loc[method,['AMI_Louvain','AMI_kmeans','AMI_HC']] = [ami_louvain,ami_kmeans,ami_hc]
    df_metrics.loc[method,['Homogeneity_Louvain','Homogeneity_kmeans','Homogeneity_HC']] = [homo_louvain,homo_kmeans,homo_hc] 
    adata.obs[['louvain','kmeans','hc']].to_csv(os.path.join(path_clusters ,method + '_clusters.tsv'),sep='\t')

ChromVAR_kmers


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 15 at resolution 1.5
step 1
got 9 at resolution 0.75
step 2
got 14 at resolution 1.125
step 3
got 10 at resolution 0.9375
step 4
got 12 at resolution 1.03125
step 5
got 13 at resolution 1.078125
ChromVAR_motifs


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 16 at resolution 1.5
step 1
got 12 at resolution 0.75
step 2
got 15 at resolution 1.125
step 3
got 15 at resolution 0.9375
step 4
got 14 at resolution 0.84375
step 5
got 14 at resolution 0.796875
step 6
got 14 at resolution 0.7734375
step 7
got 13 at resolution 0.76171875
cisTopic


...

step 0
got 32 at resolution 1.5
step 1
got 19 at resolution 0.75
step 2
got 10 at resolution 0.375
step 3
got 16 at resolution 0.5625
step 4
got 13 at resolution 0.46875
Cicero_pca


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 33 at resolution 1.5
step 1
got 20 at resolution 0.75
step 2
got 14 at resolution 0.375
step 3
got 7 at resolution 0.1875
step 4
got 9 at resolution 0.28125
step 5
got 13 at resolution 0.328125
SCRAT_pca


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 27 at resolution 1.5
step 1
got 15 at resolution 0.75
step 2
got 9 at resolution 0.375
step 3
got 11 at resolution 0.5625
step 4
got 13 at resolution 0.65625


In [9]:
df_metrics.to_csv(path_metrics+'clustering_scores.csv')

In [10]:
df_metrics

,ARI_Louvain,ARI_kmeans,ARI_HC,AMI_Louvain,AMI_kmeans,AMI_HC,Homogeneity_Louvain,Homogeneity_kmeans,Homogeneity_HC
ChromVAR_kmers,0.362156,0.318281,0.322602,0.547156,0.508931,0.505412,0.529241,0.494062,0.499065
ChromVAR_motifs,0.273587,0.234159,0.23283,0.467077,0.381515,0.389418,0.452938,0.378984,0.374202
cisTopic,0.326831,0.311433,0.334379,0.594191,0.548624,0.584576,0.562274,0.528385,0.572242
SnapATAC,0.31158,0.315666,0.238986,0.58319,0.538058,0.570649,0.549473,0.526652,0.526307
SCRAT_motifs,0.150932,0.077564,0.076397,0.299833,0.175588,0.174113,0.30107,0.17731,0.17448
BROCKMAN,0.229773,0.108314,0.0950577,0.402846,0.245153,0.216923,0.403183,0.246948,0.218872
Cusanovich2018,0.314571,0.312622,0.394549,0.588781,0.532839,0.579267,0.562294,0.520058,0.575403
Control,0.332837,0.0310206,0.0323049,0.548613,0.17679,0.18392,0.54098,0.145575,0.146171
GeneScoring,0.0740742,0.01888,0.0301842,0.159352,0.115906,0.178871,0.151916,0.0956696,0.143932
Scasat,0.358806,0.107472,0.121207,0.554397,0.33932,0.343008,0.552355,0.32181,0.327857
